# Example: Dataset CEAP-360VR with pandas and scikit-learn

This notebook loads and preprocesses the dataset `CEAP-360VR` [GitHub repo](https://github.com/luiseduve/CEAP-360VR-Dataset) described in the paper:

*CEAP-360VR: A Continuous Physiological and Behavioral Emotion Annotation Dataset for 360 VR Videos* [(DOI)](10.1109/TMM.2021.3124080)

*Description:* 

1. A class was created to load the individual Json files in a structured way through the index file `data_tree_index.json`. Similarly, demographics and video stimuli information are stored in two .csv files. The `Frame` data was used as main data source.
2. The sampling frequency is different among data modalities, they were normalized to 30Hz for all videos (Video1 was at 25Hz). Moreover, we loaded `Raw` IBI to generate new signals `IBI_R_Peaks` indicating with a 1 when a heart-rate beat was detected. This information is useful for HRV analysis.
3. Finally, the dataset is combined in a single dataframe containing all data @30Hz, without missing values and with target class labels to be used in classification tasks.

In [ ]:
import ceap_loader

# Libs for data manipulation
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt

---
## Setup

In [ ]:
# All the files generated from this notebook are in a subfolder with this name
STR_DATASET = "ceap_example/"

In [ ]:
def gen_path_temp(filename, subfolders="", extension=".csv"):
    # Function to generate temporary files easier
    TEMP_FOLDER_NAME = "./temp/"
    return ceap_loader.generate_complete_path(filename, \
                                        main_folder=TEMP_FOLDER_NAME, \
                                        subfolders=STR_DATASET+subfolders, \
                                        file_extension=extension)

---
# 1) Loading the dataset as CSV
---

In [ ]:
# Define the root folder of the dataset with respect to the notebook
dataset_root_folder = "../../../CEAP-360VR/"
print(dataset_root_folder)

The class `DatasetCEAP()` generates three files in the same folder of the dataset:
1. `data_tree_index.json`: The index containing the relative paths of the data files, grouped per data type (*Annotations, Behavior, Physio*), processing level (*Raw, Transformed, Frame*) and participant (*From 1 to 32*).
2. `demographics_info_summary.csv`: A table that summarizes, per participant, the demographic information, subjective ratings from the questionnaires, and when the participant watched each of the 8 stimuli videos.
3. `video_info_summary.csv`: A table describing the metadata from the video as in the folder `1_Stimuli`.

In [ ]:
data_manager = ceap_loader.DatasetCEAP(dataset_root_folder)

In [ ]:
# Access the stimuli data
data_manager.stimuli

In [ ]:
# Access the demographics dataframe
data_manager.demographics.head(2)

## Plotting data from one participant

In [ ]:
# Parameters of data to load
pid = 1         # 1-32
typ = "Physio"  # ["Annotations", "Behavior", "Physio"]
prep = "Frame"    # ["Raw", "Transformed", "Frame"]

# Load data 
data_loaded = data_manager.load_data_from_participant(pid,typ,prep)
data_loaded.head()

In [ ]:
# Plots all data from a participant, dividing per videoId and data type.
ceap_loader.plot_all_data_from_participant(data_loaded)

## Plotting and saving to PNG file

In [ ]:
# Load data from a new participant
data_loaded = data_manager.load_data_from_participant(5,"Annotations","Raw")

# Create the plot
ceap_loader.plot_all_data_from_participant(data_loaded)

# Save the plot in a custom folder (in this case "./temp/")
save_path_plot = gen_path_temp(f"plot_test",subfolders="plots/",extension=".png")
fig = plt.gcf()
fig.savefig(save_path_plot, dpi=300)

### Plot the data from all file in the dataset

**Uncomment if needed** The cell below takes **>2 hours** plotting the whole dataset, per loaded file.

In [ ]:
# ## Generate plots per data type and to visualize all the data per participant
# for typ in data_manager.LIST_DATA_TYPES:
#     for prep in data_manager.LIST_PROCESSING_LEVELS:
#         for pid in range(1,33):
#             data_loaded = data_manager.load_data_from_participant(pid,typ,prep)
#             ceap_loader.plot_all_data_from_participant(data_loaded)
#             save_path_plot = gen_path_temp(f"{prep}/Participant{pid}_{typ}",subfolders="plots/",extension=".png")
#             fig = plt.gcf()
#             fig.savefig(save_path_plot, dpi=200)
#             plt.close()

## Generate a CSV with data of interest

In the example below, we load the data from all participants, all data types, and processing level `Frame`. 

Creating a CSV file for the whole dataset produces a CSV file of `~800MB`.

In [ ]:
# Participants IDS
PARTICIPANTS_IDS = np.arange(1,33)
# Load data Annotations, Behavior, and Physio
DATA_GROUPS = data_manager.LIST_DATA_TYPES
# Load the Raw, Transformed, or Frame (resampled) data processing
DATA_PROCESSING_LEVELS = data_manager.LIST_PROCESSING_LEVELS
print(f"DATA_GROUPS={DATA_GROUPS}, DATA_PROCESSING_LEVELS={DATA_PROCESSING_LEVELS}")

In [ ]:
# Load or create dataframe with statistics of initial dataset
df_ceap = None

# Load all data resampled by frame
for pid in PARTICIPANTS_IDS:     # Which participants to load
    for dttype in DATA_GROUPS:   # Which data type to load
        for prep in ["Frame"]:   # Which processing level
            df_single_file = data_manager.load_data_from_participant(pid, dttype, prep)
            df_ceap = df_single_file if (df_ceap is None) else pd.concat([df_ceap, df_single_file], axis=0)

In [ ]:
df_ceap[ df_ceap["data_type"]=="Physio" ].isna().sum(axis=0)

Note that the `IBI` data is not sampled at the same frequency than the other physiological signals. Therefore, there will be missing values if you load `Physio` data.

To load a cleaner version of the data, use the parameter `clean_physio = True` in the function `load_data_from_participant()`.

In [ ]:
# Where the compiled dataset will be stored
DATASET_POSTPROCESSED_FILENAME = gen_path_temp("Dataset_CEAP", extension=".csv")
# Load or create dataframe with statistics of initial dataset
df_ceap = None
try:
    df_ceap = pd.read_csv(DATASET_POSTPROCESSED_FILENAME)
    print("Data loaded from file")
except:
    print("Creating file")
    # Load all data resampled by frame
    for pid in PARTICIPANTS_IDS:     # Which participants to load
        for dttype in DATA_GROUPS:   # Which data type to load
            for prep in ["Frame"]:   # Which processing level
                df_single_file = data_manager.load_data_from_participant(pid, dttype, prep, clean_physio=True)
                df_ceap = df_single_file if (df_ceap is None) else pd.concat([df_ceap, df_single_file], axis=0)
        
    # Saving .csv
    df_ceap.to_csv(DATASET_POSTPROCESSED_FILENAME, index=False)

print(f"\n\tFinished creating files {DATASET_POSTPROCESSED_FILENAME}")

In [ ]:
# The physiological features do not have missing values
df_ceap[ df_ceap["data_type"]=="Physio" ].isna().sum(axis=0)

In [ ]:
# The columns with missing values can be dismissed to load the corresponding data type
df_ceap[ df_ceap["data_type"]=="Physio" ].dropna(axis=1, how="all")

In [ ]:
# Another example showing only the annotations
df_ceap[ df_ceap["data_type"]=="Annotations" ].dropna(axis=1, how="all")

---
# 2. Preprocessed data
---

Returns the data from `Frame` with the following modifications:
  - the data from the `VideoID=1` was upsampled from 25 to 30Hz, to make consistent the FPS with the other stimuli videos `VideoID=2..8`.
  - Raw `IBI` was loaded to calculate HRV at 30Hz, and attached to the previous dataset.

In [ ]:
# The dictionary below can be used to recover the column names per data type
COLS_PER_DATA_TYPE = {
            'Annotations': ['Valence', 'Arousal'],
            'Behavior': ['HM_Pitch', 'HM_Yaw', 'EM_Pitch', 'EM_Yaw', 'LEM_Pitch', 'LEM_Yaw', 'REM_Pitch', 'REM_Yaw', 'LPD_PD', 'RPD_PD'],
            'Physio': ['ACC_ACC_X', 'ACC_ACC_Y', 'ACC_ACC_Z', 'SKT_SKT', 'EDA_EDA', 'BVP_BVP', 'HR_HR', 'IBI_R_Peaks']
        }

# Array with main column names in the dataset. Used to filter main columns in the dataset
participant_colname = data_manager.K_PARTICIPANT
ts_colname = data_manager.K_TIMESTAMP
video_colname = data_manager.K_VIDEO
basic_cols = ["data_type","processing_level", participant_colname, video_colname, ts_colname]

# Constant sampling frequency to be applied to data from Video1 and to transform IBI to peaks.
RESAMPLING_FREQUENCY = 30       # What is the sampling frequency of the peaks array?

### Supersample `Frame` data from VideoID=1

In [ ]:
df_ceap = ceap_loader.resample_dataframes_from_video1(df_ceap, ts_colname=ts_colname)
df_ceap.shape

In [ ]:
# All the timestamps have the same 1800 timestamps, for 60 seconds of data @ 30Hz
timestamps_reference = df_ceap.TimeStamp.unique()
print(timestamps_reference)
timestamps_reference.shape

In [ ]:
df_ceap.columns

### Convert Raw IBI to R-peaks array and attach to dataset

Next, we load the raw IBI data to extract a time series @ 30Hz that contains when a heart beat (or IBI peak) occurred

In [ ]:
# Load all raw IBI to generate HRV data
data_IBI_raw = None
for pid in PARTICIPANTS_IDS:
    df_single_file = data_manager.load_data_from_participant(pid,"Physio","Raw")
    data_IBI_raw = df_single_file if (data_IBI_raw is None) else pd.concat([data_IBI_raw, df_single_file], axis=0)

In [ ]:
# Name of the column containing IBI data (this column will be removed and replaced by R-peaks)
ibi_colname = "IBI_IBI"
r_peaks_colname = "IBI_R_Peaks" # Name that will be used after transforming IBI into R-peaks

In [ ]:
def extract_peaks_from_IBI(df, FS = 30, ibi_colname="IBI_IBI", output_colname="IBI_R_Peaks"):
    """
    Given a dataframe `df` containing irregular physiological 
    features from interbeat intervals with column name:`IBI_IBI`.
    This function returns another dataframe containing 60 seconds of
    data at the same sampling rate than the rest of the dataset
    preprocessed as `Frame`.

    This series contains the position of the `peaks` as `1`, and
    the rest of the array contains zeros. The returned dataframe can be
    directly used directly in neurokit2 package to extract HRV features:
     - `neurokit2.hrv(peaks, sampling_rate=FS)`
    """
    # The first IBI allows to regenerate a new peak right after the first IBI
    first_beat_time = df.index[0]
    first_beat_time = first_beat_time - df.iloc[0][0]
    if(first_beat_time>=0):
        df.loc[first_beat_time] = 0
    df.sort_index(inplace=True)
    
    # Generate an zero-array that will contain the R-peaks as 1's at a specific sampling frequency `FPS`
    MAXIMUM_TIME_SECS = 60
    ts_index_resampled = np.linspace(0, MAXIMUM_TIME_SECS, 60 * FS) # The way used by the authors of the dataset. I would use `np.arange(0,60,1/FS)`
    df_peaks = pd.DataFrame(data=np.zeros(ts_index_resampled.size, dtype=int), index=ts_index_resampled, columns=["IBI_IBI"])

    # Match the IBI times to the closest timestamp in the array containing the peaks
    closest_times_to_peaks = df_peaks.index.get_indexer(df.index.values, method="nearest")
    closest_index_values = df_peaks.index[closest_times_to_peaks] # Get index values from the positions
    df_peaks.loc[closest_index_values] = 1

    # The dataframe needs to be called `R_Peaks` to extract HRV with neurokit
    df_peaks = df_peaks.rename({ibi_colname:output_colname}, axis=1)
    
    return df_peaks

In [ ]:
# Where the compiled dataset will be stored
DATASET_POSTPROCESSED_WITH_RPEAKS_FILENAME = gen_path_temp("Dataset_CEAP_replacing_IBI_with_RPeaks", extension=".csv")

try:
    df_ceap_with_Rpeaks = pd.read_csv(DATASET_POSTPROCESSED_WITH_RPEAKS_FILENAME)
    print("Data loaded from file")
except:
    print("Creating file")
    df_ceap_with_Rpeaks = df_ceap

    # Add empty R_Peaks to the whole dataset
    df_ceap_with_Rpeaks[r_peaks_colname] = np.nan

    # Iterate over participants and videos to add the respective R_peaks
    for pid in np.sort(df_ceap_with_Rpeaks.ParticipantID.unique()):
        for vid in np.sort(df_ceap_with_Rpeaks.VideoID.unique()):
            #######
            # Query to filter subset of IBI data
            Q = ( (data_IBI_raw.ParticipantID == pid) 
                & (data_IBI_raw.VideoID == vid)
                & (data_IBI_raw.data_type == "Physio"))
            # Extract the R_peaks from the corresponding Raw IBI
            data_single_instance = data_IBI_raw[Q][[ts_colname, ibi_colname]].set_index(ts_colname).dropna(axis=0, how="all")

            # If contains the column, and the column has data
            if(ibi_colname in data_single_instance.columns and data_single_instance[ibi_colname].size > 0):
                data_peaks_resampled = extract_peaks_from_IBI(data_single_instance,
                                                        FS=RESAMPLING_FREQUENCY,
                                                        ibi_colname=ibi_colname,
                                                        output_colname=r_peaks_colname)
            else:
                # Dataframe full of zeros but without peaks, to compensate for those samples without IBI data.
                data_peaks_resampled = pd.DataFrame({
                    ts_colname: timestamps_reference,
                    r_peaks_colname: np.zeros(timestamps_reference.size, dtype=int),
                }).set_index(ts_colname)

            #######
            # Replace the relevant subsection of the postprocessed data
            # Query to filter subset of big dataframe
            Q = ((df_ceap_with_Rpeaks.ParticipantID == pid)
                & (df_ceap_with_Rpeaks.VideoID == vid)
                & (df_ceap_with_Rpeaks.data_type == "Physio"))
            idx_to_replace = df_ceap_with_Rpeaks[Q].index
            df_ceap_with_Rpeaks.loc[idx_to_replace, r_peaks_colname ] = data_peaks_resampled[r_peaks_colname].values
            print(f"P{pid} - V{vid} - #R-Peaks:{data_peaks_resampled[r_peaks_colname].values.sum()}")

    ## Saving .csv
    df_ceap_with_Rpeaks.to_csv( DATASET_POSTPROCESSED_WITH_RPEAKS_FILENAME, index=False)

In [ ]:
df_ceap_with_Rpeaks.head()

Example on how the peaks can be used in the feature extraction stage to calculate HRV with the package Neurokit

In [ ]:
data_IBI_one_participant = df_ceap_with_Rpeaks[ (df_ceap_with_Rpeaks["data_type"] == "Physio") & \
                                                (df_ceap_with_Rpeaks["ParticipantID"] == 1) & \
                                                (df_ceap_with_Rpeaks["VideoID"] == 3) \
                                              ].dropna(axis=1, how="all")
data_IBI_one_participant

In [ ]:
import neurokit2 as nk
x=data_IBI_one_participant["IBI_R_Peaks"]
hrv_indices = nk.hrv(x, sampling_rate=RESAMPLING_FREQUENCY, show=True)
hrv_indices

---
# 3. Organize the dataset ready for feature extraction
---

The last example shows how to organize the dataset for classification tasks:

1. Remove NaN by merging the time-series per their respective `data_group`. They already have the same `ParticipantID`, `VideoID` and `TimeStamp`, thus it's easy to remove the column that indicates the type of data (*Annotations, Behavior, Physio*) so that the whole dataframe does not contain missing values.
2. In this case, we define the class labels in high/low arousal/valence according to the video's reference V-A levels: `[HAHL, HALV, LAHV, LA,LV]`. However, the same procedure can be used to label the target classes in other ways.

In [ ]:
dgroup_colname = "data_type"        # Existing column to be removed
class_label_colname = "class_VA"    # Class column name to be created

# These columns are used as index to join df
basic_colnames = [participant_colname, video_colname, ts_colname]
basic_colnames

In [ ]:
# Mapping used according to the paper's information in Table 1
# doi: 10.1109/TMM.2021.3124080
MAPPING_VIDEO_TO_CLASS = {
    1: "HVHA",
    2: "HVLA",
    3: "LVHA",
    4: "LVLA",
    5: "HVHA",
    6: "HVLA",
    7: "LVHA",
    8: "LVLA",
}
# Create a function from the dictionary to apply on the final array
mapper_videoid_to_classes = np.vectorize(MAPPING_VIDEO_TO_CLASS.get)

In [ ]:
# Where the compiled dataset will be stored
DATASET_POSTPROCESSED_WITHOUT_NAN = gen_path_temp("Dataset_CEAP_postprocessed", extension=".csv")

# Load or create dataframe with statistics of initial dataset
dataset_postprocessed_no_nan = None

try:
    dataset_postprocessed_no_nan = pd.read_csv(DATASET_POSTPROCESSED_WITHOUT_NAN)
    print("Data loaded from file")
except:
    print("Creating file")

    # Delete preprocessing level info (Full of labels saying `Frame`)
    df_ceap_with_Rpeaks = df_ceap_with_Rpeaks.drop(["processing_level"], axis=1)

    # Merge data from different groups to remove Nan values
    for pid in np.sort(df_ceap_with_Rpeaks[participant_colname].unique()):
        for vid in np.sort(df_ceap_with_Rpeaks[video_colname].unique()):
            # Stores the different data groups per time-series instance.
            df_instance = None
            for dg in DATA_GROUPS:
                print(f"P{pid} V{vid} G:{dg}")
                Q = ( (df_ceap_with_Rpeaks[participant_colname] == pid) 
                    & (df_ceap_with_Rpeaks[video_colname] == vid)
                    & (df_ceap_with_Rpeaks[dgroup_colname] == dg))
                
                selection_idx = df_ceap_with_Rpeaks[Q].index
                data_per_group = df_ceap_with_Rpeaks.loc[selection_idx].copy()

                # Load the data get the relevant columns that do not contain missing values
                data_per_group.drop(dgroup_colname, axis=1, inplace=True)
                data_per_group.set_index(basic_colnames, inplace=True)
                data_per_group.dropna(axis=1, how="all", inplace=True)

                # Add specific data group to time series
                df_instance = data_per_group if (df_instance is None) else df_instance.join(data_per_group)

            # Add joined dataset to general one
            df_instance.reset_index(inplace=True)
            dataset_postprocessed_no_nan = df_instance if (dataset_postprocessed_no_nan is None) else pd.concat([dataset_postprocessed_no_nan, df_instance], axis=0, ignore_index=True)

print("\tEnd")

# Map each video to the corresponding Class label
video_id_array = dataset_postprocessed_no_nan[video_colname]
dataset_postprocessed_no_nan[class_label_colname] = mapper_videoid_to_classes(video_id_array)


In [ ]:
# Target array (y)
video_id_array

In [ ]:
# Input data (X)
dataset_postprocessed_no_nan

In [ ]:
# Check for missing values
dataset_postprocessed_no_nan.isna().sum(axis=0)

Following steps may involve the transformation of each time series into tabular form with overlapping windows, or directly apply time-series classifiers...

In [ ]:
# Examples to work with the data as MultiIndex (easier to apply feature extraction with rolling windows)
dataset_postprocessed_no_nan.set_index(["ParticipantID","VideoID","TimeStamp"])

In [ ]:
print('End of notebook')